In [4]:
from NeuralNetwork import NeuralNetwork, train_nn, performance, split
from _functions import *
import numpy as np
from time import time


# Get a list of all the client's models
client_models = get_dirlist("models")
client_models.remove("server")

# Find the "servers" weights
# This, by default is the first trained model that a client spits out
serverWIH= np.load("models/" + client_models[0] +"/wih.npy")
serverWHO = np.load("models/" + client_models[0]+ "/who.npy")

# Combine/Merge all the models
for model in client_models:
    
    clientWIH = np.load("models/" + model +"/wih.npy")
    clientWHO = np.load("models/" + model +"/who.npy")
    
    serverWIH = np.mean( np.array([ serverWIH, clientWIH ]), axis=0 )
    serverWHO = np.mean( np.array([ serverWHO, clientWHO ]), axis=0 )
    
    # number of input, hidden and output nodes
    input_nodes = 784
    hidden_nodes = 200
    output_nodes = 10
    learning_rate = 0.1

    # Create the neural network object
    n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
    n.update_info(serverWIH, serverWHO)
    
    print("Server performance after merge: " + str(performance(n)))



Server performance after merge: 0.9174
Server performance after merge: 0.9251
Server performance after merge: 0.9266
Server performance after merge: 0.9309
Server performance after merge: 0.9311
Server performance after merge: 0.9298
Server performance after merge: 0.9331
Server performance after merge: 0.9291
Server performance after merge: 0.933
Server performance after merge: 0.9324
Server performance after merge: 0.9307
Server performance after merge: 0.9331
Server performance after merge: 0.9305
Server performance after merge: 0.93
Server performance after merge: 0.9293
Server performance after merge: 0.9231
